##### Quantum Data Science 2023/2024
### Lecture 3 - Distance between quantum states and entanglement

<!-- no toc -->
### Contents 

1. [Important functions and libraries](#important)
2. [SWAP Test](#swaptest)
4. [EXERCISE 1 - Inversion test](#inversiontest)
5. [EXERCISE 2 - Hadamard test](#hadamardtest)
5. [EXERCISE 3 - Trace distance](#tracedistance)
6. [EXERCISE 4 - Meyer-Wallach measure](#meyerwallach)

#### 1. Libraries and functions <a id="important"></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np

##### Function for executing a quantum circuit 

In [2]:
from qiskit import *
from qiskit.primitives import Sampler,Estimator
import numpy as np

def execute_circuit(qc, shots=1024, seed=None, binary=False , primitive="sampler", observable=None):
    
    if primitive == "estimator":
        options = {"shots": shots, "seed": seed}
        estimator = Estimator(options=options)
    
        job = estimator.run(qc, observable)
        result = job.result()  
        
        return result
    
    elif primitive == "sampler":
        options = {"shots": shots, "seed": seed}
        sampler = Sampler(options=options)
    
        job = sampler.run(qc)
        result = job.result()  
        
        probability_dictionary = result.quasi_dists[0]

        if binary:
            return probability_dictionary.binary_probabilities()
        else:
            return probability_dictionary
    

#### 2. SWAP Test <a id="swaptest"></a>

<p align="center">
  <img width="500" height="250" src="images/swaptestt.png">
</p>

The swap test can be used to estimate the overlap between arbitrary pure states $|\psi\rangle$ and $|\phi\rangle$. The probability of measuring the ancilla in state $|0\rangle$ is given by: 
$$ P(0) = \frac{1}{2} + \frac{1}{2} |\langle \psi | \phi \rangle |^2$$ 

Thus, the overlap (or fidelity) between the states is given by: 

$$|\langle \psi | \phi \rangle |^2 = 2P(0) - 1$$


### <span style="color: red;">EXERCISE 1</span> - Inversion test <a id="inversiontest"></a>

<p align="center">
  <img width="450" height="200" src="images/inversion_test.png"> 
</p>

The inversion test can also be used to estimate the overlap between two pure quantum states with $n$ qubits by preparing the unitary $U_{\psi}$ and the transpose conjugated unitary $U_{\phi}^{\dagger}$ in the same set of $n$ qubits. The overlap is the probability of measuring the all-zero state. 

Prove analytically that the overlap $|\langle \phi | \psi \rangle|^2$ is given by the probability of the all-zero state. 

Implement the inversion test for arbitrary n qubit states.

In [15]:
def inversion_test(psi, phi, shots=1024):
    overlap = 0

    n = psi.num_qubits
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr)

    qc.append(psi,qr)
    qc.append(phi.inverse(),qr)

    qc.measure_all()

    prob_dict = execute_circuit(qc)

    overlap = prob_dict.get(0,0)
    
    return overlap

Compute the overlap between the states $|GHZ\rangle = \frac{1}{\sqrt{2}}\biggl( |000\rangle + |111\rangle \biggr)$ and the W state $|W\rangle = \frac{1}{\sqrt{3}}\biggl( |001\rangle + |010\rangle + |100\rangle \biggr)$.

<p align="center">
  <img width="450" height="200" src="images/wstate.png"> 
</p>

In [16]:
### YOUR CODE HERE ###

qr_psi = QuantumRegister(3)
psi = QuantumCircuit(qr_psi)

psi.h(qr_psi[0])
psi.cx(qr_psi[0],qr_psi[1])
psi.cx(qr_psi[0],qr_psi[2])


qr_phi = QuantumRegister(3)
phi = QuantumCircuit(qr_phi)

phi.ry(2*np.arccos(1/np.sqrt(3)),qr_phi[0])

phi.ch(qr_phi[0],qr_phi[1])
phi.cx(qr_phi[1],qr_phi[2])
phi.cx(qr_phi[0],qr_phi[1])
phi.x(qr_phi[0])

inversion_test(psi,phi)


0

### <span style="color: red;">EXERCISE 2</span> - Hadamard test <a id="hadamardtest"></a>

<p align="center">
  <img width="800" height="250" src="images/hadamardtest.png"> 
</p>

The Hadamard test can be used to estimate the real part of the inner product between states $|\psi\rangle$ and $|\phi\rangle$. We use the standard swap test but prepare the states conditionally on the state of the ancilla. Measuring the probability of the ancilla being in the state 0, reveals the real part of the inner product. If we prepare the ancilla in the state $|K\rangle = \frac{1}{\sqrt{2}} (|0\rangle - i|1\rangle)$ and apply the same process, we get the imaginary part of the inner product.

Implement the hadamard test. 

we can turn any quantum circuit psi = QuantumCircuit() to a controlled gate

```python
psi = QuantumCircuit()
psi = psi.to_gate().control(1,ctrl_state='0')
```
where we have 1 control bit in the state 0




In [53]:
def hadamard_test(psi, phi, imaginary=False):

    n = psi.num_qubits
    qr = QuantumRegister(n+1)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr,cr)

    psi = psi.to_gate().control(1,ctrl_state='1')
    phi = phi.to_gate().control(1,ctrl_state='0')

    if not imaginary:
        qc.h(qr[0])
    else:
        qc.x(qr[0])
        qc.h(qr[0])
        qc.s(qr[0])

    
    qc.append(psi,qr)
    qc.append(phi,qr)

    qc.h(qr[0])

    qc.measure(qr[0],cr)

    prob_dict = execute_circuit(qc)

    if imaginary:
        return 1 - 2 * prob_dict.get(0,0)
    else:
        return 2 * prob_dict.get(0,0) - 1




Test the hadamard test for the GHZ states as in the exercise 4

In [57]:
### YOUR CODE HERE ###

qr_psi = QuantumRegister(3)
psi = QuantumCircuit(qr_psi)

psi.h(qr_psi[0])
psi.cx(qr_psi[0],qr_psi[1])
psi.cx(qr_psi[0],qr_psi[2])


qr_phi = QuantumRegister(3)
phi = QuantumCircuit(qr_phi)

phi.ry(2*np.arccos(1/np.sqrt(3)),qr_phi[0])

phi.ch(qr_phi[0],qr_phi[1])
phi.cx(qr_phi[1],qr_phi[2])
phi.cx(qr_phi[0],qr_phi[1])
phi.x(qr_phi[0])

hadamard_test(psi,phi,imaginary=False)


0.001953125

### <span style="color: red;">EXERCISE 3</span> - Trace distance <a id="tracedistance"></a>

The fidelity between quantum states $|\langle \psi | \phi \rangle|^2$ only works for the case when both states are pure states. When we have generalized mixed states we need other distance measures. One such measure is the trace distance: 

$$T(\rho, \sigma) = \frac{1}{2} Tr| \rho - \sigma | = \frac{1}{2} \sum_{i}| \lambda_i |$$

where $\rho$ and $\sigma$ are the density matrices associated with the quantum states and $|\lambda_i|$ is the absolute value of the eigenvalue $i$.



In [60]:
from qiskit.quantum_info import DensityMatrix
DensityMatrix([1,0])

DensityMatrix([[1.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j]],
              dims=(2,))


In [ ]:
def trace_distance(psi, phi):

    ### YOUR CODE HERE ###
    
    pass

Test for arbitrary quantum states 

In [1]:
### YOUR CODE HERE ###

### <span style="color: red;">EXERCISE 4</span> - Meyer-Wallach measure <a id="meyerwallach"></a>

The Meyer-wallach measure is a technique developed for quantifying the entangling capability of an arbitrary quantum circuit. It can be obtained as:

$$ Q = 2 - \frac{2}{n} \sum_{k=0}^{n-1} Tr(\rho_k^2) $$ 

where $\rho_k$ is the state of the $k^\text{th}$ qubit after tracing out the rest of the system. 

In [61]:
from qiskit.quantum_info import partial_trace
#partial_trace(state_vector, list_of_qubits_to_be_traced)
partial_trace([1,0,0,0], [0])

DensityMatrix([[1.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j]],
              dims=(2,))


In [72]:
def meyer_wallach(qc, n_qubits):

    result = 2
    
    for k in range(n_qubits):
        result -= 2/n_qubits * partial_trace(DensityMatrix(qc),[k])

    return result

Measure the entanglement of the W state and the GHZ state of three qubits

In [73]:
### YOUR CODE HERE ###

qr_psi = QuantumRegister(3)
psi = QuantumCircuit(qr_psi)

psi.h(qr_psi[0])
psi.cx(qr_psi[0],qr_psi[1])
psi.cx(qr_psi[0],qr_psi[2])


qr_phi = QuantumRegister(3)
phi = QuantumCircuit(qr_phi)

phi.ry(2*np.arccos(1/np.sqrt(3)),qr_phi[0])

phi.ch(qr_phi[0],qr_phi[1])
phi.cx(qr_phi[1],qr_phi[2])
phi.cx(qr_phi[0],qr_phi[1])
phi.x(qr_phi[0])
meyer_wallach(phi,3)


TypeError: unsupported operand type(s) for -=: 'int' and 'DensityMatrix'